In [23]:
'''
        COMMON SCRIPT
        This section shall be the same for all prompts!
        
        terminal: ollama run deepseek-r1:1.5b
'''

from ollama import chat
from ollama import ChatResponse
import pandas as pd
import sys
import numpy as np
import os
import sys
from collections import defaultdict
import json
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.chat_parser import TestCaseChatParser
#MODEL = 'deepseek-r1:1.5b'
MODEL = 'deepseek-r1:7b'
#MODEL = 'deepseek-r1:8b'
#MODEL = 'deepseek-r1:14b'
#MODEL = 'deepseek-r1:32b'
#MODEL = 'deepseek-r1:70b'
#MODEL = 'deepseek-r1:671b'

DATA_PATH = "C:/Users/alexs/Documents/Studium/Informatik/Seminar/RBST-prompt-engineering/datasets/dronology/dronology_with_id.csv"

OPTIONS ={ # https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values
        "num_ctx": 10000,
        "seed": 0,
        "num_predict": -1,
        "top_k": 40,
        "top_p": 0.8,
        "min_p": 0.0,
        "repeat_last_n": 64,
        "temperature": 0.8,
        "repeat_penalty": 0.7,
        "mirostat": 2,
        "mirostat_tau": 0.5,
        "mirostat_eta": 0.1,
        }

# Load the CSV into a pandas DataFrame
df = pd.read_csv(DATA_PATH)
requirements = df['RequirementText']
ids = df['issueid']


In [22]:
'''
    Explore dronologydataset01.json dataset
'''

def count_issue_ids_by_prefix(json_data):
    # Initialize counters
    counts = defaultdict(int)
    total_ids = 0
    re_children = defaultdict(int)
    
    # Process each entry in the JSON data
    entries = json_data['entries']
    for entry in entries:
        if 'issueid' in entry:
            issue_id = entry['issueid']
            total_ids += 1
            
            # Check for each prefix
            if isinstance(issue_id, str):
                if issue_id.startswith("RE"):
                    counts["RE"] += 1
                    if (len(entry['children']) > 0):
                        re_children[issue_id] = entry['children']['refinedby']
                elif issue_id.startswith("DD"):
                    counts["DD"] += 1
                elif issue_id.startswith("ST"):
                    counts["ST"] += 1
    
    return counts, total_ids, re_children

with open("C:/Users/alexs/Documents/Studium/Informatik/Seminar/RBST-prompt-engineering/datasets/dronology/dronologydataset01.json", "r", encoding='utf-8') as f:
    data = json.load(f)

counts, total, re_children = count_issue_ids_by_prefix(data)

print("Counts by prefix:")
for prefix, count in counts.items():
    print(f"{prefix}: {count}")
    
print("Req children:")
for prefix, re_children in re_children.items():
    print(f"{prefix}: {re_children}")


Counts by prefix:
DD: 211
RE: 99
ST: 63
Req children:
RE-766: ['DD-439']
RE-759: ['DD-362', 'DD-468', 'DD-498', 'DD-536', 'DD-537', 'DD-538', 'DD-540']
RE-754: ['DD-755', 'DD-756', 'DD-757']
RE-751: ['DD-752', 'DD-753']
RE-746: ['DD-747', 'DD-748']
RE-741: ['DD-742', 'DD-743', 'DD-744', 'DD-745']
RE-739: ['DD-740']
RE-736: ['DD-737']
RE-729: ['DD-727', 'DD-728', 'DD-730', 'DD-731', 'DD-732', 'DD-733', 'DD-734', 'DD-735', 'DD-768']
RE-722: ['DD-723', 'DD-724', 'DD-763']
RE-714: ['DD-361', 'DD-715', 'DD-716']
RE-709: ['DD-354', 'DD-710', 'DD-711', 'DD-712', 'DD-713']
RE-706: ['DD-707']
RE-702: ['DD-703', 'DD-704', 'DD-705']
RE-701: ['DD-445']
RE-698: ['DD-699']
RE-695: ['DD-696', 'DD-697']
RE-693: ['DD-694']
RE-691: ['DD-692']
RE-689: ['DD-229', 'DD-690']
RE-681: ['DD-113', 'DD-682', 'DD-683']
RE-679: ['DD-168']
RE-677: ['DD-678']
RE-672: ['DD-673', 'DD-674', 'DD-675', 'DD-676', 'DD-767']
RE-670: ['DD-671']
RE-668: ['DD-669', 'DD-762']
RE-666: ['DD-667']
RE-664: ['DD-421', 'DD-665']
RE-6

In [24]:
'''
    ### PROMPT 0 ###
    ### Zero shot learning ###
    ### + directional stimulus prompting ###
    ### Base line ###
'''

parser_0 = TestCaseChatParser()

FILE_NAME_RESULTS = "results_chat_prompt0_zero.json"

empty_template = """{
    "Requirement": "",
    "testObjective": "",
    "preconditions": [],
    "testSteps": [],
    "expectedResult": ""
}"""

JSON_MSG = [
    {"role": "system", "content": empty_template}
]

OPTIONS ={ # https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values
        "num_ctx": 10000,
        "seed": 0,
        "num_predict": -1,
        "top_k": 40,
        "top_p": 0.9,
        "min_p": 0.1,
        "repeat_last_n": 64,
        "temperature": 0.8,
        "repeat_penalty": 0.1,
        "mirostat": 2,
        "mirostat_tau": 0.5,
        "mirostat_eta": 0.1,
        }

'''
    REQUIREMENT LOOP
'''
for tc_id, req in zip(requirements, ids):
    msg = [{"role": "user", "content": "Generate json test description using this template: " + empty_template + " For the following requirement: " + req}]

    print("PROMPT: ")
    print(msg)
    response:ChatResponse = chat(
        model=MODEL, 
        format='json',
        options=OPTIONS,
        #stop=["."], # stop condition, e.g. "." stops generation after one sentence
        messages=msg
    )
    print("RESPONSE: ")
    print(response.message.content)
    parser_0.add_entry(tc_id, response.message.content, msg)
    print("==================================")

parser_0.save(FILE_NAME_RESULTS)


PROMPT: 
[{'role': 'user', 'content': 'Generate json test description using this template: {\n    "Requirement": "",\n    "testObjective": "",\n    "preconditions": [],\n    "testSteps": [],\n    "expectedResult": ""\n} For the following requirement: RE-766'}]
RESPONSE: 
{ "Requirement": "RE-766", "testObjective": "To test the functionality of the new user authentication feature introduced in version 1.2 of the application.", "preconditions": ["The application must be built with the new authentication library version 2.3.1", "The user database schema must be updated to support the new authentication tokens", "A test environment with sufficient user permissions must be provided for the test team to perform the test."], "testSteps": ["1. Verify that the authentication header is present in the application's response when a user logs in.", "2. Log in to the application using the new authentication token.", "3. Verify that the system logs a successful authentication and grants access to the

In [ ]:
'''
    ### PROMPT 1 ###
    ### PERSONAS ###
    ### + directional stimulus prompting ###

'''
parser_1 = TestCaseChatParser()

FILE_NAME_RESULTS = "results_chat_prompt1_persona.json"
PERSONA_MSG = [
    {"role": "system", "content": "You are an expert software tester for Unmanned Aerial Systems (UAS). Reply using json.\n\nEXAMPLE INPUT REQUIREMENT:\nWhen a user cancels a pending flight plan the RouteAssignmentUI shall notify Dronology.\n\nEXAMPLE JSON OUTPUT:\n{\n    \"Requirement\": \"When a user cancels a pending flight plan the RouteAssignmentUI shall notify Dronology.\",\n    \"testObjective\": \"Verify that when a user cancels a pending flight plan the RouteAssignmentUI notifies Dronology.\",\n    \"preconditions\": [\n        \"The application must be running.\",\n        \"One or more flight plans are pending.\"\n    ],\n    \"testSteps\": [\n        \"1. Add one or more flight plans.\",\n        \"2. As a user cancel a flight plan.\",\n        \"3. Wait until RouteAssignmentUI notifies Dronology.\"\n    ],\n    \"expectedResult\": \"Flight plan is canceled and Dronology received notification from RouteAssignmentUI.\"\n}"},
]


'''
    REQUIREMENT LOOP
'''
for tc_id, req in enumerate(requirements):
    msg = [{"role": "user", "content": "Generate json test description for the following requirement. " + req}]
    msg = PERSONA_MSG + msg

    print("PROMPT: ")
    print(msg)
    response:ChatResponse = chat(
        model=MODEL, 
        format='json',
        options=OPTIONS,
        #stop=["."], # stop condition, e.g. "." stops generation after one sentence
        messages=msg
    )
    print("RESPONSE: ")
    print(response.message.content)
    parser_1.add_entry(tc_id, response.message.content, msg)
    print("==================================")

parser_1.save(FILE_NAME_RESULTS)

PROMPT: 
[{'role': 'system', 'content': 'You are an expert software tester for Unmanned Aerial Systems (UAS). Reply using json.\n\nEXAMPLE INPUT REQUIREMENT:\nWhen a user cancels a pending flight plan the RouteAssignmentUI shall notify Dronology.\n\nEXAMPLE JSON OUTPUT:\n{\n    "Requirement": "When a user cancels a pending flight plan the RouteAssignmentUI shall notify Dronology.",\n    "testObjective": "Verify that when a user cancels a pending flight plan the RouteAssignmentUI notifies Dronology.",\n    "preconditions": [\n        "The application must be running.",\n        "One or more flight plans are pending."\n    ],\n    "testSteps": [\n        "1. Add one or more flight plans.",\n        "2. As a user cancel a flight plan.",\n        "3. Wait until RouteAssignmentUI notifies Dronology."\n    ],\n    "expectedResult": "Flight plan is canceled and Dronology received notification from RouteAssignmentUI."\n}'}, {'role': 'user', 'content': 'Generate json test description for the fo

In [ ]:
'''
    ### POROMPT 2 ###
    ### TREE OF THOUGHT ###
    ### + directional stimulus prompting ###
    complex tasks tell it multi experts conversation
    therefore the system runs it multiple times and compares results
'''
parser_2 = TestCaseChatParser()

FILE_NAME_RESULTS = "results_chat_prompt2_tot.json"
TREE_OF_THOUGHT = [
    {"role": "system", "content": "Imagine three expert software testers for Unmanned Aerial Systems (UAS). All experts receive the same software requirement and define a test description. All experts will write down 1 step of their thinking then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes they are wrong at any point they leave."},
    {"role": "system", "content": "Respond in json.\n\nEXAMPLE INPUT REQUIREMENT:\nWhen a user cancels a pending flight plan the RouteAssignmentUI shall notify Dronology.\n\nEXAMPLE JSON OUTPUT:\n{\n    \"Requirement\": \"When a user cancels a pending flight plan the RouteAssignmentUI shall notify Dronology.\",\n    \"testObjective\": \"Verify that when a user cancels a pending flight plan the RouteAssignmentUI notifies Dronology.\",\n    \"preconditions\": [\n        \"The application must be running.\",\n        \"One or more flight plans are pending.\"\n    ],\n    \"testSteps\": [\n        \"1. Add one or more flight plans.\",\n        \"2. As a user cancel a flight plan.\",\n        \"3. Wait until RouteAssignmentUI notifies Dronology.\"\n    ],\n    \"expectedResult\": \"Flight plan is canceled and Dronology received notification from RouteAssignmentUI.\"\n}"},
]


'''
    REQUIREMENT LOOP
'''
for tc_id, req in enumerate(requirements):
    msg = [{"role": "user", "content": "The experts shall generate json test description for the following requirement. " + req}]
    msg = TREE_OF_THOUGHT + msg

    print("PROMPT: ")
    print(msg)
    response:ChatResponse = chat(
        model=MODEL, 
        format='json',
        options=OPTIONS,
        #stop=["."], # stop condition, e.g. "." stops generation after one sentence
        messages=msg
    )
    print("RESPONSE: ")
    print(response.message.content)
    parser_2.add_entry(tc_id, response.message.content, msg)
    print("==================================")

parser_2.save(FILE_NAME_RESULTS)


PROMPT: 
[{'role': 'system', 'content': 'Imagine three expert software testers for Unmanned Aerial Systems (UAS). All experts receive the same software requirement and define a test description. All experts will write down 1 step of their thinking then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes they are wrong at any point they leave.'}, {'role': 'system', 'content': 'Respond in json.\n\nEXAMPLE INPUT REQUIREMENT:\nWhen a user cancels a pending flight plan the RouteAssignmentUI shall notify Dronology.\n\nEXAMPLE JSON OUTPUT:\n{\n    "Requirement": "When a user cancels a pending flight plan the RouteAssignmentUI shall notify Dronology.",\n    "testObjective": "Verify that when a user cancels a pending flight plan the RouteAssignmentUI notifies Dronology.",\n    "preconditions": [\n        "The application must be running.",\n        "One or more flight plans are pending."\n    ],\n    "testSteps": [\n        "1. Add one or more flig

In [ ]:
'''
    ### POROMPT 3 ###
    ### PROVIDING CONTEXT ###
    ### + directional stimulus prompting (implicit through template) ###
'''

"""
The Dronology Architecture is designed to deliver a multi-platform solution.  It  is designed around the concept of a Dronology Core with Services middleware that support UI development and Groundstation middleware that supports communication with diverse UAS.  Here we describe each core component and provide links to other pages containing further details.
Dronology Core provides features for coordinating the flights of multiple sUAS in a shared airspace.  This includes centralized collision avoidance, fleet management, internal simulator, and management of vehicle status.  Click here to learn more about key components in the Dronology core.
Dronology services are used by various clients to construct user interfaces.  The services can be used to perform tasks such as upload flight plans, activate UASs, observe and visualize UAS flights, and register equipment to individual UAS.
Vaadin is an open-source web framework that supports development of rich user interfaces.  It has a server-side architecture, which allows most of the logic to run on the server, while Ajax technology on the client side ensures a rich and highly interactive UI.  In Dronology, we run Vaadin on TomCat — which can be deployed either centrally or locally.  The web-based approach enables our goal of platform portability.
The ground station middleware provides a bridge between the Dronology core and a diverse set of ground stations.  It is designed to allow future developers to build their own ground stations.  The middleware allows registration of a new ground station.  It passes directives from the Dronology core to UAVs registered through their respective ground stations, and forwards UAV data, such as their GPS coordinates from the ground station back to the the core.
Groundstations are responsible for communicating directly with the physical or externally simulated UAVs.  Our current ground station is developed in Python using Dronekit Python — and is capable of communicating with UAVs that run ArduPilot.
The runtime monitoring component allows constraints to be defined and various values to be monitored.  The monitored variables include UAV specific values such as coordinates, velocity, and battery power, as well as external variables read from sensors on the UAV. 
"""

parser_3 = TestCaseChatParser()

FILE_NAME_RESULTS = "results_chat_prompt3_context_and_tot.json"
CONTEXT = [
    {"role": "system", "content": "The Dronology Architecture is designed to deliver a multi-platform solution.  It  is designed around the concept of a Dronology Core with Services middleware that support UI development and Groundstation middleware that supports communication with diverse UAS.  Here we describe each core component and provide links to other pages containing further details. \
Dronology Core provides features for coordinating the flights of multiple sUAS in a shared airspace.  This includes centralized collision avoidance, fleet management, internal simulator, and management of vehicle status.  Click here to learn more about key components in the Dronology core. \
Dronology services are used by various clients to construct user interfaces.  The services can be used to perform tasks such as upload flight plans, activate UASs, observe and visualize UAS flights, and register equipment to individual UAS. \
Vaadin is an open-source web framework that supports development of rich user interfaces.  It has a server-side architecture, which allows most of the logic to run on the server, while Ajax technology on the client side ensures a rich and highly interactive UI.  In Dronology, we run Vaadin on TomCat — which can be deployed either centrally or locally.  The web-based approach enables our goal of platform portability. \
The ground station middleware provides a bridge between the Dronology core and a diverse set of ground stations.  It is designed to allow future developers to build their own ground stations.  The middleware allows registration of a new ground station.  It passes directives from the Dronology core to UAVs registered through their respective ground stations, and forwards UAV data, such as their GPS coordinates from the ground station back to the the core. \
Groundstations are responsible for communicating directly with the physical or externally simulated UAVs.  Our current ground station is developed in Python using Dronekit Python — and is capable of communicating with UAVs that run ArduPilot. \
The runtime monitoring component allows constraints to be defined and various values to be monitored.  The monitored variables include UAV specific values such as coordinates, velocity, and battery power, as well as external variables read from sensors on the UAV. "},
    {"role": "system", "content": "Imagine three expert software testers for Unmanned Aerial Systems (UAS). All experts receive the same software requirement and define a test description. All experts will write down 1 step of their thinking then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes they are wrong at any point they leave."},
    {"role": "system", "content": "Respond in json.\n\nEXAMPLE INPUT REQUIREMENT:\nWhen a user cancels a pending flight plan the RouteAssignmentUI shall notify Dronology.\n\nEXAMPLE JSON OUTPUT:\n{\n    \"Requirement\": \"When a user cancels a pending flight plan the RouteAssignmentUI shall notify Dronology.\",\n    \"testObjective\": \"Verify that when a user cancels a pending flight plan the RouteAssignmentUI notifies Dronology.\",\n    \"preconditions\": [\n        \"The application must be running.\",\n        \"One or more flight plans are pending.\"\n    ],\n    \"testSteps\": [\n        \"1. Add one or more flight plans.\",\n        \"2. As a user cancel a flight plan.\",\n        \"3. Wait until RouteAssignmentUI notifies Dronology.\"\n    ],\n    \"expectedResult\": \"Flight plan is canceled and Dronology received notification from RouteAssignmentUI.\"\n}"},
]


'''
    REQUIREMENT LOOP
'''
for tc_id, req in enumerate(requirements):
    msg = [{"role": "user", "content": "The experts shall generate json test description for the following requirement. " + req}]
    msg = CONTEXT + msg

    print("PROMPT: ")
    print(msg)
    response:ChatResponse = chat(
        model=MODEL, 
        format='json',
        options=OPTIONS,
        #stop=["."], # stop condition, e.g. "." stops generation after one sentence
        messages=msg
    )
    print("RESPONSE: ")
    print(response.message.content)
    parser_3.add_entry(tc_id, response.message.content, msg)
    print("==================================")

parser_3.save(FILE_NAME_RESULTS)

PROMPT: 
[{'role': 'system', 'content': 'The Dronology Architecture is designed to deliver a multi-platform solution.  It  is designed around the concept of a Dronology Core with Services middleware that support UI development and Groundstation middleware that supports communication with diverse UAS.  Here we describe each core component and provide links to other pages containing further details. Dronology Core provides features for coordinating the flights of multiple sUAS in a shared airspace.  This includes centralized collision avoidance, fleet management, internal simulator, and management of vehicle status.  Click here to learn more about key components in the Dronology core. Dronology services are used by various clients to construct user interfaces.  The services can be used to perform tasks such as upload flight plans, activate UASs, observe and visualize UAS flights, and register equipment to individual UAS. Vaadin is an open-source web framework that supports development of

In [ ]:
'''
    ### POROMPT 4 ###

    preparsing of requirement + tot

'''

parser_4 = TestCaseChatParser()

FILE_NAME_RESULTS = "results_chat_prompt4_preparsing_and_tot_reqfirst.json"
TREE_OF_THOUGHT = [
    {"role": "system", "content": "Imagine three expert software testers for Unmanned Aerial Systems (UAS). All experts receive the same software requirement and define a test description. All experts will write down 1 step of their thinking then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes they are wrong at any point they leave."},
    {"role": "system", "content": "Respond in json.\n\nEXAMPLE INPUT REQUIREMENT:\nWhen a user cancels a pending flight plan the RouteAssignmentUI shall notify Dronology.\n\nEXAMPLE JSON OUTPUT:\n{\n    \"Requirement\": \"When a user cancels a pending flight plan the RouteAssignmentUI shall notify Dronology.\",\n    \"testObjective\": \"Verify that when a user cancels a pending flight plan the RouteAssignmentUI notifies Dronology.\",\n    \"preconditions\": [\n        \"The application must be running.\",\n        \"One or more flight plans are pending.\"\n    ],\n    \"testSteps\": [\n        \"1. Add one or more flight plans.\",\n        \"2. As a user cancel a flight plan.\",\n        \"3. Wait until RouteAssignmentUI notifies Dronology.\"\n    ],\n    \"expectedResult\": \"Flight plan is canceled and Dronology received notification from RouteAssignmentUI.\"\n}"},
]
REQUIREMENTS_CONTEXT = "These are all the software requirements for the system:\n\n"
REQUIREMENTS_CONTEXT += "\n".join([f"{i+1}. {req}" for i, req in enumerate(requirements)])
REQ_CONT_LIST = [{"role": "system", "content": REQUIREMENTS_CONTEXT}]

'''
    REQUIREMENT LOOP
'''
for tc_id, req in enumerate(requirements):
    msg = [{"role": "user", "content": "The experts shall generate json test description for the following requirement. " + req}]
    msg = REQ_CONT_LIST + TREE_OF_THOUGHT + msg

    print("PROMPT: ")
    print(msg)
    response:ChatResponse = chat(
        model=MODEL, 
        format='json',
        options=OPTIONS,
        #stop=["."], # stop condition, e.g. "." stops generation after one sentence
        messages=msg
    )
    print("RESPONSE: ")
    print(response.message.content)
    parser_4.add_entry(tc_id, response.message.content, msg)
    print("==================================")

parser_4.save(FILE_NAME_RESULTS)

PROMPT: 
[{'role': 'system', 'content': "These are all the software requirements for the system:\n\n1. The MapComponent shall support different types of map layers (e.g.  terrain  satellite)\n2. The MissionPlanner shall execute flight plans for multiple UAVs concurrently.\n3. The GCS shall transmit the UAV's properties to the GCSMiddleware\n4. The GCS shall transmit the UAV's current location to the GCSMiddleware\n5. The GCS shall report newly connected UAVs to the GCSMiddleware.\n6. The GCS shall assign a message frequency for all UAVs.\n7. When the GCS receives a UAV command from the GCSMiddleware  the GCS shall forward the command to the respective UAV.\n8. When the connection to the GCS from the GCSMiddleware is lost  the GCSMiddleware shall notify Dronology and deactivate all associated UAVs.\n9. The GCSMiddleware shall forward commands sent from Dronology to a GCS\n10. The GCSMiddleware shall handle state messages   monitoring messages  and mode change messages  received from an 

In [ ]:
'''
    ### POROMPT 5 ###

using model:
    🌋 LLaVA: Large Language and Vision Assistant
    -> multimodal context
    parse architecture picture

'''

In [ ]:
'''
    ### PROMPT 6 ###
        use Requirements + Design Specifications == refined definition of original requirement.
'''


